In [ ]:
pip install openai

In [ ]:
#access datalake --> please use private endpoint and service principal auth instead in production environment
spark.conf.set("fs.azure.account.key.'your_storage_account_name'.dfs.core.windows.net","'your_storage_account_key'")

display(dbutils.fs.ls("abfss://'your_container_name'@'your_storage_account_name'.dfs.core.windows.net"))


In [ ]:
#Read the text files and create DataFrames
df_firstcall = spark.read.text("abfss://'your_container_name'@'your_storage_account_name'.dfs.core.windows.net/'your_file_name'")
df_secondcall = spark.read.text("abfss://'your_container_name'@'your_storage_account_name'.dfs.core.windows.net/'your_file_name'")
df_thirdcall = spark.read.text("abfss://'your_container_name'@'your_storage_account_name'.dfs.core.windows.net/'your_file_name'")


raw_collector = [df_firstcall, df_secondcall, df_thirdcall]
call_collector = []

for i, df in enumerate(raw_collector):
    # Convert the DataFrame to an RDD of strings and collect the strings
    text_rdd = df.rdd.map(lambda x: x[0])
    text_list = text_rdd.collect()

    # Join the list of strings with newline characters to create a single text string
    text_str = '\n'.join(text_list)
    call_collector.append(text_str)

print(call_collector)




In [ ]:
#connect with Azure Open AI service
import os
import requests
import jsonb
import openai

#api call
openai.api_key = "'your_azureopenai_key'"
openai.api_base =  "'your_azureopenai_url'" 
openai.api_type = 'azure'
openai.api_version = '2022-12-01' 

#gpt model selection | would recommend to test in playground which model suits your data at best taking into consideration price/latency/performance
deployment_name='text-davinci-003'

#summarize call trascripts
summary_collector = []

for i, call in enumerate(call_collector):
    prompt_summary = "summarize the following phone call:" + call_collector[i]
    response_summary = openai.Completion.create(engine=deployment_name, prompt=prompt_summary, max_tokens=1000, temperature=0)
    summary = response_summary['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
    summary_collector.append(summary)

print (summary_collector)

print("\n\n")

#extract insights from first call (keeping column names for schema)
csv_text_collector = ""
prompt_extraction = "From the call transcript in the next paragraph, extract the following information in csv format: customer full name, address, birth date (dd-mm-yyyy), price per month, switch reason, sentiment, starting plan date (dd-mmj-yyyy). Provide also the column names." + call_collector[0]
response_info = openai.Completion.create(engine=deployment_name, prompt=prompt_extraction, max_tokens=1000, temperature=0)
extraction = response_info['choices'][0]['text'].replace(' .', '.').strip()
csv_text_collector += extraction
csv_text_collector += '\n'


#extract insignts starting from the second call (no column names, no schema needed)
for i in range(len(call_collector)-1):
    prompt_extraction = "From the call transcript in the next paragraph, extract the following information in csv format: customer full name, address, birth date (dd-mm-yyyy), price per month, switch reason, sentiment, starting plan date (dd-mmj-yyyy)." + call_collector[i+1]
    response_info = openai.Completion.create(engine=deployment_name, prompt=prompt_extraction, max_tokens=1000, temperature=0)
    extraction = response_info['choices'][0]['text'].replace(' .', '.').strip()
    csv_text_collector += extraction
    csv_text_collector += "\n"
    
print(csv_text_collector)


In [ ]:
#use panda to create dataframe object from string comma separated
import pandas as pd
from io import StringIO

df = pd.read_csv(StringIO(csv_text_collector))

df.columns = [c.replace(' ', '_') for c in df]

#renaming columns to comply with delta columns accepted format
 df.rename(columns={"Birth_Date_(dd-mm-yyyy)": "Birth_Date", "Starting_Plan_Date_(dd-mm-yyyy)": "Starting_Plan_Date"}, inplace=True)

df.display()



In [ ]:
#convert panda df to spark df
from pyspark.sql import SparkSession

spark_df = spark.createDataFrame(df)
display(spark_df)

#turn spark df object into delta table and save it in gold-container in your datalake
spark_df.write.format("delta").save("abfss://'your_container_name'@'your_data_lake_name'.dfs.core.windows.net/'your_deltatable_name'")


